In [1]:
import pandas as pd

In [2]:
game_df = pd.read_csv('data/celticsbruins_games.csv')
rides_df = pd.read_csv('data/UberLyft_weather.csv')

In [3]:
# convert each timestamp from the rides and the games into datetime
game_df['time_stamp'] = pd.to_datetime(game_df['time_stamp'])
game_df['stop_time'] = pd.to_datetime(game_df['stop_time'])
rides_df['time_stamp'] = pd.to_datetime(rides_df['time_stamp'])

game_df.head()

,source,type,time_stamp,stop_time
0,North Station,NBA/Celtics,2018-11-30 19:00:00,2018-11-30 21:30:00
1,North Station,NBA/Celtics,2018-12-06 20:00:00,2018-12-06 22:30:00
2,North Station,NBA/Celtics,2018-12-10 19:30:00,2018-12-10 22:00:00
3,North Station,NBA/Celtics,2018-12-14 19:00:00,2018-12-14 21:30:00
4,North Station,NHL/Bruins,2018-11-29 19:00:00,2018-11-29 21:20:00


In [4]:
# sorts game by datetime in ascending order
game_df = game_df.sort_values(by=['time_stamp'])

game_df.head()

,source,type,time_stamp,stop_time
4,North Station,NHL/Bruins,2018-11-29 19:00:00,2018-11-29 21:20:00
0,North Station,NBA/Celtics,2018-11-30 19:00:00,2018-11-30 21:30:00
5,North Station,NHL/Bruins,2018-12-01 19:00:00,2018-12-01 21:20:00
1,North Station,NBA/Celtics,2018-12-06 20:00:00,2018-12-06 22:30:00
6,North Station,NHL/Bruins,2018-12-08 19:00:00,2018-12-08 21:20:00


In [5]:
# merge game dataframe based on ride timestamp occuring two hours within the start and stop times of the game
comb_df = pd.merge_asof(rides_df, game_df, on='time_stamp', by='source', tolerance=pd.Timedelta('2 hour')).drop('stop_time', axis = 1).reset_index(drop=True)
game_df.rename(columns={"time_stamp": "start_time", "stop_time" : "time_stamp"}, inplace=True)
comb_df = pd.merge_asof(comb_df, game_df, on='time_stamp', by='source', tolerance=pd.Timedelta('2 hour')).drop('start_time', axis = 1).reset_index(drop=True)

# merge game dataframe for both source and destination of the game
game_df.rename(columns={"start_time": "time_stamp", "time_stamp" : "stop_time", "source" : "destination"}, inplace=True)

comb_df = pd.merge_asof(comb_df, game_df, on='time_stamp', by='destination', tolerance=pd.Timedelta('2 hour')).drop('stop_time', axis = 1).reset_index(drop=True).rename(columns = {'type_x' : 'x', 'type_y':'y'})
game_df.rename(columns={"time_stamp": "start_time", "stop_time" : "time_stamp"}, inplace=True)
comb_df = pd.merge_asof(comb_df, game_df, on='time_stamp', by='destination', tolerance=pd.Timedelta('2 hour')).drop('start_time', axis = 1).reset_index(drop=True)

comb_df.head()

,Unnamed: 0,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,...,temp,clouds,pressure,rain,humidity,wind,x,y,type_x,type_y
0,0,1.51,Uber,2018-11-26 05:26:40,Theatre District,Haymarket Square,15.0,1.0,5321c595-ddbe-4f24-b128-cf0ffb2944fb,6c84fd89-3f11-4782-9b50-97c468b19529,...,40.64,0.93,1013.78,0.0,0.92,3.01,NaN,NaN,NaN,NaN
1,1,2.32,Uber,2018-11-26 05:26:40,North End,Back Bay,14.0,1.0,74a108a3-8d5f-4445-8f9b-dd8a0f8e5ae5,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,...,40.64,0.92,1013.76,0.0,0.92,3.02,NaN,NaN,NaN,NaN
2,2,1.18,Uber,2018-11-26 05:26:40,West End,South Station,10.5,1.0,ad8c44f5-9ace-4e30-aa72-9f44e2e2304b,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,...,40.56,0.93,1013.77,0.0,0.93,3.17,NaN,NaN,NaN,NaN
3,3,1.18,Uber,2018-11-26 05:26:40,West End,South Station,10.5,1.0,a363f700-6009-4e51-953a-38176cf958ec,55c66225-fbe7-4fd5-9072-eab1ece5e23e,...,40.56,0.93,1013.77,0.0,0.93,3.17,NaN,NaN,NaN,NaN
4,4,2.32,Uber,2018-11-26 05:26:40,North End,Back Bay,9.5,1.0,574c4736-7681-48da-be82-7e5ce23537a0,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,...,40.64,0.92,1013.76,0.0,0.92,3.02,NaN,NaN,NaN,NaN


In [6]:
# for all game matches, replace value with 1
comb_df = comb_df.replace(['NHL/Bruins', 'NBA/Celtics'], 1)
comb_df = comb_df.fillna(0) # for non game matches, replace value with 0

comb_df.head()

,Unnamed: 0,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,...,temp,clouds,pressure,rain,humidity,wind,x,y,type_x,type_y
0,0,1.51,Uber,2018-11-26 05:26:40,Theatre District,Haymarket Square,15.0,1.0,5321c595-ddbe-4f24-b128-cf0ffb2944fb,6c84fd89-3f11-4782-9b50-97c468b19529,...,40.64,0.93,1013.78,0.0,0.92,3.01,0.0,0.0,0.0,0.0
1,1,2.32,Uber,2018-11-26 05:26:40,North End,Back Bay,14.0,1.0,74a108a3-8d5f-4445-8f9b-dd8a0f8e5ae5,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,...,40.64,0.92,1013.76,0.0,0.92,3.02,0.0,0.0,0.0,0.0
2,2,1.18,Uber,2018-11-26 05:26:40,West End,South Station,10.5,1.0,ad8c44f5-9ace-4e30-aa72-9f44e2e2304b,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,...,40.56,0.93,1013.77,0.0,0.93,3.17,0.0,0.0,0.0,0.0
3,3,1.18,Uber,2018-11-26 05:26:40,West End,South Station,10.5,1.0,a363f700-6009-4e51-953a-38176cf958ec,55c66225-fbe7-4fd5-9072-eab1ece5e23e,...,40.56,0.93,1013.77,0.0,0.93,3.17,0.0,0.0,0.0,0.0
4,4,2.32,Uber,2018-11-26 05:26:40,North End,Back Bay,9.5,1.0,574c4736-7681-48da-be82-7e5ce23537a0,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,...,40.64,0.92,1013.76,0.0,0.92,3.02,0.0,0.0,0.0,0.0


In [7]:
# create dummy variable column for the game matches and non-matches
comb_df['Sports Occurence'] = comb_df['x'] + comb_df['y'] + comb_df['type_x'] + comb_df['type_y']

# remove unnecessary columns from final dataframe
comb_df = comb_df.drop(['Unnamed: 0', 'x', 'y', 'type_x', 'type_y'], axis = 1)

comb_df.head()

,distance,cab_type,time_stamp,destination,source,price,surge_multiplier,id,product_id,name,temp,clouds,pressure,rain,humidity,wind,Sports Occurence
0,1.51,Uber,2018-11-26 05:26:40,Theatre District,Haymarket Square,15.0,1.0,5321c595-ddbe-4f24-b128-cf0ffb2944fb,6c84fd89-3f11-4782-9b50-97c468b19529,Black,40.64,0.93,1013.78,0.0,0.92,3.01,0.0
1,2.32,Uber,2018-11-26 05:26:40,North End,Back Bay,14.0,1.0,74a108a3-8d5f-4445-8f9b-dd8a0f8e5ae5,6f72dfc5-27f1-42e8-84db-ccc7a75f6969,UberXL,40.64,0.92,1013.76,0.0,0.92,3.02,0.0
2,1.18,Uber,2018-11-26 05:26:40,West End,South Station,10.5,1.0,ad8c44f5-9ace-4e30-aa72-9f44e2e2304b,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV,40.56,0.93,1013.77,0.0,0.93,3.17,0.0
3,1.18,Uber,2018-11-26 05:26:40,West End,South Station,10.5,1.0,a363f700-6009-4e51-953a-38176cf958ec,55c66225-fbe7-4fd5-9072-eab1ece5e23e,UberX,40.56,0.93,1013.77,0.0,0.93,3.17,0.0
4,2.32,Uber,2018-11-26 05:26:40,North End,Back Bay,9.5,1.0,574c4736-7681-48da-be82-7e5ce23537a0,9a0e7b09-b92b-4c41-9779-2ad22b4d779d,WAV,40.64,0.92,1013.76,0.0,0.92,3.02,0.0
